# Stock Track Portfolio Analysis

### Group # 9 - Landen Fogle, Mia Siner, Emmett Jhonson, ?
 Our portfolio strategy: Large-Cap Growth

 Benchmark Inded: SWLGX | Schwab U.S. Large-Cap Growth Index Fund



## Section 1: Portfolio Generation Code
In part by ChatGPT

In [47]:
from matplotlib import pyplot as plt
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display

class StockPortfolio:
    def __init__(self):
        self.positions = []
        self.cash = 1000000.00  # Starting cash balance
        self.portfolio_value = self.cash
        self.benchmark = "SWLGX"  # Example benchmark
        self.week_number = 0
        self.week_end_date = None
        self.weekly_data = []

    def add_position(self, ticker, shares, purchase_date, purchase_price):
        stock_data = yf.Ticker(ticker)
        try:
            stock_info = stock_data.info
            beta = stock_info.get('beta', 0.0)
            company_name = stock_info.get('longName', ticker)
            industry = stock_info.get('industry', 'Unknown')

            position = {
                'ticker': ticker,
                'company_name': company_name,
                'industry': industry,
                'shares': shares,
                'purchase_date': purchase_date,
                'purchase_price': purchase_price,
                'end_of_last_week_price': None,
                'end_of_week_price': None,
                'beta': beta,
                'performance': None,
                'total_value': None,
                'weight': None
            }
            self.positions.append(position)
            self.cash -= shares * purchase_price
            self.update_portfolio_value()
        except Exception as e:
            print(f"Error retrieving data for {ticker}: {e}")

    def update_positions(self, week_end_date):
        self.week_end_date = week_end_date
        for position in self.positions:
            stock_data = yf.Ticker(position['ticker'])
            
            history = stock_data.history(start=week_end_date, end=(datetime.strptime(week_end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d"))
            if not history.empty:
                end_of_week_price = history['Close'].iloc[0]
            else:
                history = stock_data.history(period='1d', end=week_end_date)
                end_of_week_price = history['Close'].iloc[-1] if not history.empty else None

            if end_of_week_price is None:
                print(f"No price data found for {position['ticker']} on or before {week_end_date}. Please check the ticker or date.")
                continue

            last_week_date = (datetime.strptime(week_end_date, "%Y-%m-%d") - timedelta(days=7)).strftime("%Y-%m-%d")
            last_week_history = stock_data.history(start=last_week_date, end=week_end_date)
            if not last_week_history.empty:
                end_of_last_week_price = last_week_history['Close'].iloc[-1]
            else:
                print(f"No price data found for {position['ticker']} during the previous week.")
                end_of_last_week_price = end_of_week_price

            position['end_of_last_week_price'] = end_of_last_week_price
            position['end_of_week_price'] = end_of_week_price
            position['performance'] = ((end_of_week_price - end_of_last_week_price) / end_of_last_week_price) * 100
            position['total_value'] = position['shares'] * end_of_week_price

        self.update_portfolio_value()

    def update_portfolio_value(self):
        self.portfolio_value = self.cash + sum(pos['total_value'] for pos in self.positions if pos['total_value'] is not None)

    def calculate_portfolio_weight(self):
        total_value = sum(pos['total_value'] for pos in self.positions if pos['total_value'] is not None) + self.cash
        for position in self.positions:
            if position['total_value']:
                position['weight'] = f"{round((position['total_value'] / total_value) * 100, 2)}%"

    def adjust_cash(self, trades_this_week):
        # Define the annual interest rate and convert it to a daily rate
        annual_interest_rate = 0.08
        daily_interest_rate = (1 + annual_interest_rate) ** (1/365) - 1

        # Assume a week has 7 days, calculate weekly interest based on the daily rate
        weekly_interest_rate = (1 + daily_interest_rate) ** 7 - 1

        # Apply interest to the remaining cash balance
        interest_earned = self.cash * weekly_interest_rate
        self.cash += interest_earned

        # Deduct commission for the trades made during the week
        commission_cost = trades_this_week * 10  # $10 per trade
        self.cash -= commission_cost

        # Update portfolio value to reflect the change in cash
        self.update_portfolio_value()

        # Print the adjustment details for transparency
        print(f"Interest earned this week: ${interest_earned:.2f}")
        print(f"Total commission deducted for {trades_this_week} trades: ${commission_cost:.2f}")
        print(f"New cash balance: ${self.cash:.2f}")

    def display_portfolio(self):
        self.calculate_portfolio_weight()
        data = []
        for pos in self.positions:
            data.append({
                'Stock ticker': pos['ticker'],
                'Purchase Date': pos['purchase_date'],
                'Company Name': pos['company_name'],
                'Industry': pos['industry'],
                'Beta': pos['beta'],
                'Purchase price $': pos['purchase_price'],
                '# Shares': pos['shares'],
                'End of last week Price': pos['end_of_last_week_price'],
                'End of week Price': pos['end_of_week_price'],
                'Performance (percentage change)': pos['performance'],
                'End of week Total value': pos['total_value'],
                'Portfolio Weight': pos['weight']
            })

        df = pd.DataFrame(data)
        display(df.style.set_table_attributes("style='display:inline'").set_caption("Stock Portfolio Weekly Summary"))

    def weekly_summary(self, week_end_date):
        self.week_number += 1
        self.update_positions(week_end_date)
        weekly_return = self.calculate_weekly_return()
        
        summary = {
            'week_number': self.week_number,
            'portfolio_value': self.portfolio_value,
            'cash': self.cash,
            'positions_value': self.portfolio_value - self.cash,
            'weekly_return': weekly_return,
            'benchmark': self.benchmark,
            'beta': np.mean([pos['beta'] for pos in self.positions if pos['beta'] is not None]),
            'week_end_date': self.week_end_date
        }
        self.weekly_data.append(summary)
        return summary

    def calculate_weekly_return(self):
        start_value = sum(pos['end_of_last_week_price'] * pos['shares'] for pos in self.positions if pos['end_of_last_week_price'] is not None)
        end_value = sum(pos['end_of_week_price'] * pos['shares'] for pos in self.positions if pos['end_of_week_price'] is not None)
        return ((end_value - start_value) / start_value) * 100 if start_value != 0 else 0
    
    def display_weekly_summary(self):
        # Ensure weekly data is available
        if not self.weekly_data:
            print("No weekly data available. Please call weekly_summary() first.")
            return

        latest_week = self.weekly_data[-1]
        
        # Calculate total portfolio return since inception
        initial_value = self.cash + sum(pos['purchase_price'] * pos['shares'] for pos in self.positions)
        total_portfolio_return = ((latest_week['portfolio_value'] - initial_value) / initial_value) * 100
        
        # Calculate the weekly portfolio return
        weekly_return = latest_week['weekly_return']

        # Fetch benchmark weekly return from Yahoo Finance
        benchmark_data = yf.Ticker(self.benchmark).history(period='1wk', end=latest_week['week_end_date'])
        if not benchmark_data.empty:
            benchmark_weekly_return = ((benchmark_data['Close'].iloc[-1] - benchmark_data['Close'].iloc[0]) / benchmark_data['Close'].iloc[0]) * 100
        else:
            benchmark_weekly_return = "N/A"

        # Prepare the data for display in a table format
        summary_data = {
            'Metric': [
                'Week #', 'Portfolio Value', 'Cash', 'Positions Value', 'End of Week Date',
                'Beta for Portfolio', 'Portfolio Total Return', 'Weekly Portfolio Return', 'Benchmark Weekly Return'
            ],
            'Value': [
                latest_week['week_number'],
                f"${latest_week['portfolio_value']:.2f}",
                f"${latest_week['cash']:.2f}",
                f"${latest_week['positions_value']:.2f}",
                latest_week['week_end_date'],
                f"{round(latest_week['beta'], 2)}",
                f"{total_portfolio_return:.2f}%",
                f"{weekly_return:.2f}%",
                f"{benchmark_weekly_return:.2f}%" if benchmark_weekly_return != "N/A" else "N/A"
            ]
        }

        # Create a DataFrame for the summary data
        summary_df = pd.DataFrame(summary_data)
        
        # Display the DataFrame as a styled table
        display(summary_df.style.set_table_attributes("style='display:inline'").set_caption("Weekly Portfolio Summary"))



# Week 1 - No Purchase
#### We had communicated issues with creating and paying for our Stock Track account with the professor. We will be starting on week two. 

In [48]:
# Create Portfolio
portfolio = StockPortfolio()
# Paid Interest
portfolio.adjust_cash(trades_this_week=0)

Interest earned this week: $1477.05
Total commission deducted for 0 trades: $0.00
New cash balance: $1001477.05


# Week 2 Initial Purchase
### Buy Long 
- MA
- ADBE
- TSLA
- UNH
- META
- GOOGL
- NVDA
- AMZN
- MSFT
- AAPl 

In [49]:
#Make Trades
portfolio.add_position('MA', 181, '2024-10-07', 514.22)
portfolio.add_position('ADBE', 177, '2024-10-07', 502.51)
portfolio.add_position('TSLA', 360, '2024-10-07', 221.98)
portfolio.add_position('UNH', 152, '2024-10-07', 573.29)
portfolio.add_position('META', 151, '2024-10-07', 577.05)
portfolio.add_position('GOOGL', 539, '2024-10-07', 164.93)
portfolio.add_position('NVDA', 720, '2024-10-07', 136.51)
portfolio.add_position('AMZN', 483, '2024-10-07', 186.96)
portfolio.add_position('MSFT', 216, '2024-10-07', 415.82)
portfolio.add_position('AAPL', 397, '2024-10-07', 231.47)

# Week 2 Buy Requierment
Catching up from week 1
## Buy Long
- NFLX
- PYPL

In [50]:
portfolio.add_position('NFLX', 30, '2024-10-08', 718.56)
portfolio.add_position('PYPL', 200, '2024-10-11', 80.68)
portfolio.adjust_cash(trades_this_week=12)
portfolio.weekly_summary('2024-10-11')
portfolio.display_portfolio()
portfolio.display_weekly_summary()

Interest earned this week: $101.00
Total commission deducted for 12 trades: $120.00
New cash balance: $68362.88


,Stock ticker,Purchase Date,Company Name,Industry,Beta,Purchase price $,# Shares,End of last week Price,End of week Price,Performance (percentage change),End of week Total value,Portfolio Weight
0,MA,2024-10-07,Mastercard Incorporated,Credit Services,1.103000,514.220000,181,498.019989,502.260010,0.851376,90909.061768,9.09%
1,ADBE,2024-10-07,Adobe Inc.,Software - Infrastructure,1.289000,502.510000,177,503.570007,495.420013,-1.618443,87689.342377,8.77%
2,TSLA,2024-10-07,"Tesla, Inc.",Auto Manufacturers,2.297000,221.980000,360,238.770004,217.800003,-8.782511,78408.001099,7.84%
3,UNH,2024-10-07,UnitedHealth Group Incorporated,Healthcare Plans,0.593000,573.290000,152,597.700012,598.049988,0.058554,90903.598145,9.09%
4,META,2024-10-07,"Meta Platforms, Inc.",Internet Content & Information,1.216000,577.050000,151,583.830017,589.950012,1.048249,89082.451843,8.91%
5,GOOGL,2024-10-07,Alphabet Inc.,Internet Content & Information,1.038000,164.930000,539,162.080002,163.240005,0.715698,87986.362961,8.8%
6,NVDA,2024-10-07,NVIDIA Corporation,Semiconductors,1.669000,136.510000,720,134.809998,134.800003,-0.007414,97056.002197,9.71%
7,AMZN,2024-10-07,"Amazon.com, Inc.",Internet Retail,1.147000,186.960000,483,186.649994,188.820007,1.162611,91200.063538,9.12%
8,MSFT,2024-10-07,Microsoft Corporation,Software - Infrastructure,0.896000,415.820000,216,415.839996,416.320007,0.115432,89925.121582,9.0%
9,AAPL,2024-10-07,Apple Inc.,Consumer Electronics,1.239000,231.470000,397,229.039993,227.550003,-0.650537,90337.351212,9.04%


SWLGX: No price data found, symbol may be delisted (period=1wk)


,Metric,Value
0,Week #,1
1,Portfolio Value,$999645.94
2,Cash,$68362.88
3,Positions Value,$931283.06
4,End of Week Date,2024-10-11
5,Beta for Portfolio,1.27
6,Portfolio Total Return,-0.18%
7,Weekly Portfolio Return,-0.64%
8,Benchmark Weekly Return,N/A


# Week 3
# Buy Long
- BUD
- DIS

In [51]:
portfolio.add_position('BUD', 300, '2024-10-16', 65.59)
portfolio.add_position('DIS', 200, '2024-10-16', 94.79)
portfolio.adjust_cash(trades_this_week=2)
portfolio.weekly_summary('2024-10-15')
portfolio.display_portfolio()
portfolio.display_weekly_summary()

Interest earned this week: $43.91
Total commission deducted for 2 trades: $20.00
New cash balance: $29751.79


,Stock ticker,Purchase Date,Company Name,Industry,Beta,Purchase price $,# Shares,End of last week Price,End of week Price,Performance (percentage change),End of week Total value,Portfolio Weight
0,MA,2024-10-07,Mastercard Incorporated,Credit Services,1.103000,514.220000,181,506.660004,507.230011,0.112503,91808.631989,9.21%
1,ADBE,2024-10-07,Adobe Inc.,Software - Infrastructure,1.289000,502.510000,177,509.649994,508.029999,-0.317864,89921.309784,9.02%
2,TSLA,2024-10-07,"Tesla, Inc.",Auto Manufacturers,2.297000,221.980000,360,219.160004,219.570007,0.187080,79045.202637,7.93%
3,UNH,2024-10-07,UnitedHealth Group Incorporated,Healthcare Plans,0.593000,573.290000,152,605.400024,556.289978,-8.111999,84556.076660,8.48%
4,META,2024-10-07,"Meta Platforms, Inc.",Internet Content & Information,1.216000,577.050000,151,590.419983,586.270020,-0.702883,88526.772949,8.88%
5,GOOGL,2024-10-07,Alphabet Inc.,Internet Content & Information,1.038000,164.930000,539,164.960007,165.460007,0.303104,89182.943619,8.94%
6,NVDA,2024-10-07,NVIDIA Corporation,Semiconductors,1.669000,136.510000,720,138.070007,131.600006,-4.686029,94752.004395,9.5%
7,AMZN,2024-10-07,"Amazon.com, Inc.",Internet Retail,1.147000,186.960000,483,187.539993,187.690002,0.079988,90654.271179,9.09%
8,MSFT,2024-10-07,Microsoft Corporation,Software - Infrastructure,0.896000,415.820000,216,419.140015,418.739990,-0.095439,90447.837891,9.07%
9,AAPL,2024-10-07,Apple Inc.,Consumer Electronics,1.239000,231.470000,397,231.300003,233.850006,1.102466,92838.452423,9.31%


SWLGX: No price data found, symbol may be delisted (period=1wk)


,Metric,Value
0,Week #,2
1,Portfolio Value,$997047.69
2,Cash,$29751.79
3,Positions Value,$967295.90
4,End of Week Date,2024-10-15
5,Beta for Portfolio,1.26
6,Portfolio Total Return,-0.44%
7,Weekly Portfolio Return,-1.24%
8,Benchmark Weekly Return,N/A
